## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import random

N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = [a + 100 for a in A]

mean_B = sum(B) / N
print(mean_B)

599.943039


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import random
import string
import pandas as pd

N = 2000000
data = {'col1': [random.randint(0, 1000) for i in range(N)],
        'col2': [random.randint(0, 1000) for i in range(N)],
        'col3': [random.randint(0, 1000) for i in range(N)],
        'col4': [random.randint(0, 1000) for i in range(N)]}

df = pd.DataFrame(data)
def random_string(leng):
    return ''.join(random.choice(string.ascii_lowercase))  #множество английских букв

df['key'] = [random_string(5) for i in range(N)]

alf_5 = df[df['key'].str.match('[a-e]{5}')]
print(df)
print(alf_5)

         col1  col2  col3  col4 key
0         167   659   822   414   m
1         313   100   469   498   x
2         707   592   733   226   c
3         878   132   832   694   c
4         299   569   521   394   e
...       ...   ...   ...   ...  ..
1999995   915   911   258   509   t
1999996   431   598   792   444   f
1999997    94   425   562   447   g
1999998   513    80   561   706   t
1999999     6   658    11   173   m

[2000000 rows x 5 columns]
Empty DataFrame
Columns: [col1, col2, col3, col4, key]
Index: []


## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.2/662.2 kB 12.6 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
recipes = pd.read_csv("recipes_sample.csv")
reviews = pd.read_csv('reviews_sample.csv')
recipes['submitted'] = recipes['submitted'].astype("datetime64[ns]")
reviews['date'] = reviews['date'].astype("datetime64[ns]")
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [ ]:
def A(df):
    total_rating = 0
    num_reviews = 0
    for index, row in df.iterrows():
        if row['date'].year == 2010:
            total_rating += row['rating']
            num_reviews += 1
    mean_rating = total_rating / num_reviews
    return mean_rating

In [ ]:
%%time
A(reviews)

CPU times: total: 8.94 s
Wall time: 8.94 s


4.4544402182900615

In [ ]:
def B(df):
    reviews_2010 = df[reviews['date'].dt.year == 2010]
    total_rating = 0
    num_reviews = 0
    for index, row in reviews_2010.iterrows():
        total_rating += row['rating']
        num_reviews += 1
    mean_rating = total_rating / num_reviews
    return mean_rating

In [ ]:
%%time
B(reviews)

CPU times: total: 812 ms
Wall time: 816 ms


4.4544402182900615

In [ ]:
def C(df):
    reviews_2010 = df[reviews['date'].dt.year == 2010]
    mean_rating = reviews_2010['rating'].mean()
    return mean_rating

In [ ]:
%%time
C(reviews)

CPU times: total: 15.6 ms
Wall time: 18.8 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [1]:
!pip install line_profiler
%load_ext line_profiler
from line_profiler import LineProfiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 9.3 MB/s eta 0:00:00


In [ ]:
%lprun -f A A(reviews) 

Timer unit: 1e-07 s

Total time: 32.766 s
File: C:\Users\eliza\AppData\Local\Temp\ipykernel_25536\1115547370.py
Function: A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def A(df):
     2         1         15.0     15.0      0.0      total_rating = 0
     3         1          8.0      8.0      0.0      num_reviews = 0
     4    126696  274783792.0   2168.8     83.9      for index, row in df.iterrows():
     5    114602   48350371.0    421.9     14.8          if row['date'].year == 2010:
     6     12094    4411054.0    364.7      1.3              total_rating += row['rating']
     7     12094     114909.0      9.5      0.0              num_reviews += 1
     8         1         17.0     17.0      0.0      mean_rating = total_rating / num_reviews
     9         1          7.0      7.0      0.0      return mean_rating

In [ ]:
%lprun -f В В(reviews)

Timer unit: 1e-07 s

Total time: 0.0214808 s
File: C:\Users\eliza\AppData\Local\Temp\ipykernel_25536\193603030.py
Function: C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def C(df):
     2         1     208034.0 208034.0     96.8      reviews_2010 = df[reviews['date'].dt.year == 2010]
     3         1       6767.0   6767.0      3.2      mean_rating = reviews_2010['rating'].mean()
     4         1          7.0      7.0      0.0      return mean_rating

In [ ]:
def B1(df):
    df=df[df['date'].dt.year==2010]
    return df['rating'].sum()/df.shape[0]

In [ ]:
%%time
B1(reviews)

CPU times: total: 15.6 ms
Wall time: 19.1 ms


4.4544402182900615

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
import pandas as pd
recipes  = pd.read_csv('recipes_sample.csv')
review  = pd.read_csv('reviews_sample.csv')
review.rename(columns = {'Unnamed: 0' : 'kjbsv'})

,kjbsv,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [ ]:

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [ ]:
%%time
long_func = get_word_reviews_count(review)

CPU times: user 21.6 s, sys: 103 ms, total: 21.7 s
Wall time: 22.7 s


Можно избавиться от повторных проходов по датафрейму, объединив два цикла в один. 

В имеющемся коде неоптимально использование двух циклов для подсчета количества отзывов, содержащих каждое слово. Также неэффективно создание списка слов для каждого отзыва, вместо того, чтобы использовать итератор. 


In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in set(words):
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for word, reviews in word_reviews.items():
        word_reviews_count[word] = len(reviews)
    return word_reviews_count

In [ ]:
%%time
fast_func = get_word_reviews_count(review)


CPU times: user 11.6 s, sys: 46.7 ms, total: 11.6 s
Wall time: 11.8 s


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
#A. Без использования векторизованных операций и методов массивов numpy и без использования numba:
def mape_a(reviews):
    rating_sum = 0.0
    count = 0
    for r in reviews:
        if r != 0:
            rating_sum += r
            count += 1
    mean_rating = rating_sum / count
    abs_diff_sum = 0.0
    for r in reviews:
        if r != 0:
            abs_diff_sum += abs(r - mean_rating) / mean_rating
    return (abs_diff_sum / count) * 100

In [ ]:
%%time
mape_a(reviews)

CPU times: total: 547 ms
Wall time: 560 ms


39.86261012569429

In [ ]:
#B. Без использования векторизованных операций и методов массивов numpy, но с использованием numba:
@numba.jit(nopython=True)
def mape_b(reviews):
    rating_sum = 0.0
    count = 0
    for r in reviews:
        if r != 0:
            rating_sum += r
            count += 1
    mean_rating = rating_sum / count
    abs_diff_sum = 0.0
    for r in reviews:
        if r != 0:
            abs_diff_sum += abs(r - mean_rating) / mean_rating
    return (abs_diff_sum / count) * 100

In [ ]:
%%time
mape_b(reviews)

CPU times: total: 281 ms
Wall time: 322 ms


39.86261012569429

In [ ]:
#C. С использованием векторизованных операций и методов массивов numpy, но без использования numba:
def mape_c(reviews):
    reviews = np.array(reviews)
    reviews = reviews[reviews != 0]
    mean_rating = reviews.mean()
    abs_diff = np.abs(reviews - mean_rating) / mean_rating
    return abs_diff.mean() * 100

In [ ]:
%%time
mape_c(reviews)

CPU times: total: 15.6 ms
Wall time: 4 ms


39.86261012571317

In [ ]:
#D. С использованием векторизованных операций и методов массивов numpy и numba:
@numba.jit(nopython=True)
def mape_d(reviews):
    return np.mean(np.abs(reviews - np.mean(reviews)) / reviews) * 100

In [ ]:
%%time
mape_d(reviews)

CPU times: total: 203 ms
Wall time: 207 ms


inf

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4